We introduce two improvements to generate better water mask inferences from Planet imagery
- First, we train a Histogram-based Gradient Boosting Classification Tree instead of a Random Forest model
- Second, we generate segments using a preprocessed NIR channel PlanetScope image (compared to using the `[green, nir, ndwi]` stack in previous notebooks)

The remaining training and inference steps remain the same. At the end of our notebook, we save our model for future use.

In [6]:
# GIS imports
import rasterio

# scikit and numpy imports
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from skimage.segmentation import felzenszwalb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from skimage import filters, exposure
from skimage.morphology import square

# misc imports
import pandas as pd
from pathlib import Path
from collections import defaultdict
import joblib
from joblib import dump
from multiprocessing import Pool
from tqdm import tqdm
import os

# local imports
from tools import (get_superpixel_stds_as_features, get_superpixel_means_as_features, 
                    get_array_from_features, reproject_arr_to_match_profile, get_segment_sizes)
from rf_funcs import calc_ndwi, calc_ndvi, return_grn_indices, return_img_bands, return_reflectance_coeffs

# for repeatability
np.random.seed(42)

In [7]:
# Set to True to re-train the model
RETRAIN_MODEL=True
EVALUATE_MODEL=True # Split available data and print model performance on test set
TEST_SET_SPLIT=0.15 # If evaluating model, specify data split for testing. train split will be 1-TEST_SET_SPLIT
UPDATE_DATABASE=True # Set to True to update validation database. When re-calculating DSWx-HLS accuracy, the model outputs generated here will be used

# BOOSTED TREES PARAMETERS
MODEL_LEARNING_RATE=0.8
MODEL_MAX_ITER=300
MODEL_L2_REGULARIZATION=0.3

# FELZENSZWALB PARAMETERS
F_SCALE = 20
F_MINSIZE = 20
F_SIGMA = 0

In [8]:
# Read the validation database
data_path = Path('../data/')
val_chips_db = data_path / 'validation_table.csv'
val_df = pd.read_csv(val_chips_db)

site_names = list(val_df['site_name'])
planet_ids = list(val_df['planet_id'])

# Extract planet IDs and associated strata
site_names_stratified = defaultdict(list)
for sn, planet_id in zip(site_names, planet_ids):
    site_names_stratified[sn[:2]].append(planet_id)

training_sites = []

# We can either use 4 chips from each strata, or ALL chips from each strata
for key in site_names_stratified.keys():
        training_sites.extend(site_names_stratified[key])

print("# of Training sites: ", len(training_sites))

# of Training sites:  52


In [9]:
# Helper function that takes a Planet image and associated labels and returns training samples to be used by our model
def process_one_site(site, denoising_weight=0.):
    current_img_path = data_path / site
    cropped_img_path = data_path / 'planet_images_cropped' / site
    
    xml_file = list(current_img_path.glob('*.xml'))[0]
    chip = list(cropped_img_path.glob(f'cropped_{site}*.tif'))[0]
    classification = list(cropped_img_path.glob(f'site_name-*.tif'))[0] 

    band_idxs = return_grn_indices(xml_file)
    coeffs = return_reflectance_coeffs(xml_file, band_idxs)
    chip_img = return_img_bands(chip, band_idxs, denoising_weight=denoising_weight)
    with rasterio.open(chip) as src_ds:
        ref_profile = src_ds.profile

    green = chip_img[0]*coeffs[band_idxs[0]]
    red = chip_img[1]*coeffs[band_idxs[1]]
    nir = chip_img[2]*coeffs[band_idxs[2]]

    nodata_mask = nir == ref_profile['nodata']

    ndwi = calc_ndwi(green, nir)
    ndvi = calc_ndvi(red, nir)

    with rasterio.open(classification) as src_ds:
        cl = src_ds.read(1)
        cl_profile = src_ds.profile

    # some classification extents are not the same as the corresponding planet chip extent
    # if they are not the same, reproject the validation data to match the profile of the planet data
    if ((ref_profile['transform'] != cl_profile['transform']) | 
        (ref_profile['width'] != cl_profile['width']) | 
        (ref_profile['height'] != cl_profile['height'])):

        cl, _ = reproject_arr_to_match_profile(cl, cl_profile, ref_profile)
        cl = np.squeeze(cl)
    
    new_img = exposure.adjust_gamma(exposure.equalize_hist(filters.scharr(nir), nbins=64), gamma=20)
    new_img[nodata_mask] = np.nan
    segments = felzenszwalb(new_img, scale=F_SCALE, min_size=F_MINSIZE, sigma=F_SIGMA)

    # create training data that includes other channels as well
    img_stack = np.stack([red, nir, green, ndwi, ndvi], axis=-1)     
    std_features = get_superpixel_stds_as_features(segments, img_stack)
    mean_features = get_superpixel_means_as_features(segments, img_stack)
    segment_sizes = get_segment_sizes(segments)

    X_temp = np.concatenate([mean_features, std_features, segment_sizes], axis = 1)

    # # We have superpixels, we now need to map each of the segments to the associated label
    # # A 0 value indicates no label for the segment
    class_features_temp = np.zeros((mean_features.shape[0], 1)) + 255

    superpixel_labels_for_land = set(np.unique(segments[cl == 0]))
    superpixel_labels_for_water = set(np.unique(segments[cl == 1]))

    intersecting_labels = superpixel_labels_for_land.intersection(superpixel_labels_for_water)
    superpixel_labels_for_land = list(superpixel_labels_for_land - intersecting_labels)
    superpixel_labels_for_water = list(superpixel_labels_for_water - intersecting_labels)

    class_features_temp[superpixel_labels_for_land] = 0
    class_features_temp[superpixel_labels_for_water] = 1

    return X_temp, class_features_temp


In [10]:
# For each of the training sites - 
# 1. Read the cropped planet image and the corresponding labeled data
# 2. Segment image to generate training samples
# 3. Append training samples to list

rf_model_folder = data_path / 'trained_model' / 'rf_model'
rf_model_folder.mkdir(exist_ok=True, parents=True)
model_path = rf_model_folder/"xgboost_model.joblib"

# Write out model parameters for future reference
if RETRAIN_MODEL:
    with open(f"{model_path}.txt", 'w') as f:
        f.write(f"MODEL_LEARNING_RATE:{MODEL_LEARNING_RATE}\n")
        f.write(f"MODEL_MAX_ITER:{MODEL_MAX_ITER}\n")
        f.write(f"MODEL_L2_REGULARIZATION:{MODEL_L2_REGULARIZATION}\n\n")

        f.write(f"FELZENSZWALB SCALE:{F_SCALE}\n")
        f.write(f"FELZENSZWALB MIN SIZE:{F_MINSIZE}\n")
        f.write(f"FELZENSZWALB SIGMA:{F_SIGMA}\n")

    features_and_labels = list(map(process_one_site, tqdm(training_sites))) 
    X = np.concatenate([x[0] for x in features_and_labels], axis=0)
    class_features = np.concatenate([x[1] for x in features_and_labels], axis=0)

    # remove all segments where label is > 1 (no data regions)
    valid_idxs = np.squeeze(class_features < 2)
    X = X[valid_idxs, :]
    class_features = class_features[valid_idxs, :]

    print("Array lengths: ", X.shape, class_features.shape)

    print("Beginning model training")
    rf = HistGradientBoostingClassifier(random_state=0, class_weight='balanced',early_stopping=False, max_iter=MODEL_MAX_ITER, learning_rate=MODEL_LEARNING_RATE, l2_regularization=MODEL_L2_REGULARIZATION) 

    if EVALUATE_MODEL:
        X, X_test, class_features, class_features_test = train_test_split(X, class_features, train_size=TEST_SET_SPLIT, random_state=42)

    # train model on all of the available data
    rf.fit(X, class_features.ravel())
    if EVALUATE_MODEL:
        # print("Model OOB score: ", rf.oob_score_)
        y_pred = rf.predict(X_test)
        print("Precision/Recall/Support: ", score(y_pred, class_features_test))
        with open(f"{model_path}.txt", 'a') as f:
            f.write(f"\n\nPrecision/Recall/Support: {score(y_pred, class_features_test)}")

    # save model weights
    dump(rf, model_path)

# Let's make inferences on the broader planet images
def generate_inference_helper(rf, img:str|Path, xml_file:str|Path, denoising_weight=0.):
    band_idxs = return_grn_indices(xml_file)
    coeffs = return_reflectance_coeffs(xml_file, band_idxs)
    
    full_img = return_img_bands(img, band_idxs, denoising_weight=denoising_weight)

    green = full_img[0]*coeffs[band_idxs[0]]
    red = full_img[1]*coeffs[band_idxs[1]]
    nir = full_img[2]*coeffs[band_idxs[2]]

    with rasterio.open(img) as ds:
        ref_profile = ds.profile

    nodata_mask = nir == ref_profile['nodata']

    ndwi = calc_ndwi(green, nir)
    ndvi = calc_ndvi(red, nir)

    print(f"Starting segmentation for {img}")
    new_img = exposure.adjust_gamma(exposure.equalize_hist(filters.scharr(nir), nbins=64), gamma=20)
    new_img[nodata_mask] = np.nan
    
    segments = felzenszwalb(new_img, scale=F_SCALE, min_size=F_MINSIZE, sigma=F_SIGMA)
    
    print(f"Segmentation complete for {img}")

    # for inference we include other channels as well
    img_stack = np.stack([red, nir, green, ndwi, ndvi], axis=-1)
    std_features = get_superpixel_stds_as_features(segments, img_stack)
    mean_features = get_superpixel_means_as_features(segments, img_stack)
    segment_sizes = get_segment_sizes(segments)

    X = np.concatenate([mean_features, std_features, segment_sizes], axis = 1)
    print(f"Starting inference for {img}")
    y = rf.predict(X)

    return get_array_from_features(segments, np.expand_dims(y, axis=1))

def generate_inference(planet_id):
    """ 
    This function takes in a planet_id and generates inferences for the overlapping planet image
    """
    data_path = Path('../data')
    
    current_img_path = data_path / planet_id
    cropped_img_path = data_path / 'planet_images_cropped' / planet_id
    xml_file = list(current_img_path.glob('*.xml'))[0]
    classification = list(cropped_img_path.glob(f'site_name-*{planet_id}*.tif'))[0]

    img = list(current_img_path.glob(f'{planet_id}*.tif'))[0]
    
    print("Test file name:", img)
    assert img.exists(), "File does not exist!!"

    # make a local copy of the model would help
    _ = os.system(f"cp {model_path} {current_img_path}")
    model_copy = current_img_path / model_path.name

    rf = joblib.load(model_copy)

    inference = generate_inference_helper(rf, img, xml_file)

    # use planet image to mask out regions of no data in the model inference
    with rasterio.open(img) as src_ds:
        nodata_mask = np.where(src_ds.read(1) == src_ds.profile['nodata'], 1, 0)
        inference[nodata_mask==1] = 255
        profile_copy = src_ds.profile
        profile_copy.update({'count':1, 'dtype':np.uint8, 'nodata':255})

        # write out model inference
        with rasterio.open(f"{classification.parent}/xgboost_classification.tif", 'w', **profile_copy) as dst_ds:
            dst_ds.write(inference.astype(np.uint8).reshape(1, *inference.shape))

    print(f"Completed inference for planet id {planet_id}")
    model_copy.unlink()

print("Generating inferences for: ", planet_ids)

_ = list(map(generate_inference, planet_ids))

  0%|          | 0/52 [00:00<?, ?it/s]/u/leffe-data2/karthikv/mambaforge-pypy3/envs/expand-validation/lib/python3.11/site-packages/scipy/ndimage/_measurements.py:641: RuntimeWarning: invalid value encountered in divide
  means = sums / counts
  2%|▏         | 1/52 [00:13<11:50, 13.93s/it]/u/leffe-data2/karthikv/mambaforge-pypy3/envs/expand-validation/lib/python3.11/site-packages/scipy/ndimage/_measurements.py:641: RuntimeWarning: invalid value encountered in divide
  means = sums / counts
  4%|▍         | 2/52 [00:27<11:26, 13.72s/it]/u/leffe-data2/karthikv/mambaforge-pypy3/envs/expand-validation/lib/python3.11/site-packages/scipy/ndimage/_measurements.py:641: RuntimeWarning: invalid value encountered in divide
  means = sums / counts
  6%|▌         | 3/52 [00:41<11:16, 13.81s/it]/u/leffe-data2/karthikv/mambaforge-pypy3/envs/expand-validation/lib/python3.11/site-packages/scipy/ndimage/_measurements.py:641: RuntimeWarning: invalid value encountered in divide
  means = sums / counts
  8%

Array lengths:  (948207, 11) (948207, 1)
Beginning model training
Precision/Recall/Support:  (array([0.99806193, 0.97661162]), array([0.99742444, 0.98230923]), array([0.99774308, 0.97945214]), array([726443,  79533]))
Generating inferences for:  ['20210903_150800_60_2458', '20210903_152641_60_105c', '20210904_093422_44_1065', '20210906_101112_28_225a', '20210909_000649_94_222b', '20210911_001230_44_2262', '20210911_005129_82_106a', '20210912_034049_22_2421', '20210912_094213_84_240f', '20210914_094548_30_2406', '20210914_103644_25_2413', '20210915_011051_87_240a', '20210915_172340_12_245f', '20210915_173832_80_2307', '20210916_010848_94_2407', '20210917_140704_93_2262', '20210917_152712_47_2457', '20210922_171337_39_2420', '20210924_000522_94_2421', '20210924_082025_48_2424', '20210924_133812_95_2420', '20210925_072712_16_2254', '20210926_020646_15_2231', '20210927_105543_66_2424', '20210928_141837_16_2407', '20210928_211311_91_2457', '20210929_073913_09_2453', '20210930_021156_60_2434

In [11]:
if UPDATE_DATABASE:
    db_df = pd.read_csv('../data/new_validation_table.csv')
    planet_ids = list(val_df['planet_id'])
    inference_path = Path('../data/planet_images_cropped')
    rf_classification_files = []
    for id in planet_ids:
        rf_classification_files.append(list((inference_path / id).glob('xgboost_classification.tif'))[0])

    db_df['rf_classification_files'] = rf_classification_files
    db_df.to_csv('../data/new_validation_table.csv', index=None)